https://python.langchain.com/docs/tutorials/chatbot/

在本教程中，我们将介绍如何设计和实现一个由LLM驱动的聊天机器人。这个聊天机器人将能够进行对话，并记住与聊天模型之前的交互。  
  
请注意，我们构建的这个聊天机器人只会使用语言模型来进行对话。还有2个相关的概念你可能会感兴趣：  
  
对话式RAG（检索增强生成）：实现在外部数据源之上的聊天机器人体验。  
Agent：构建一个可以执行动作的聊天机器人。  
  
本教程只涵盖基础知识，这些知识对于上述两个更高级的主题很有帮助。  

这一章节涉及的内容包括：
1. 保存消息历史
2. prompt模板
3. 管理消息历史
4. streaming

In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [2]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(
    #model_name="deepseek-r1:32b",
    model_name="qwen2",
    openai_api_base="http://127.0.0.1:11434/v1",
    openai_api_key="EMPTY",
    streaming=False
)

In [3]:
"""
首先，让我们直接使用该模型。ChatModels是LangChain Runnables的实例，这意味着它们提供了一个与之交互的标准接口。要简单地调用模型，我们可以将消息列表传递给.invoke方法。这种方法允许用户以一种标准化的方式与模型进行互动，使得调用模型变得直接且易于操作。这种方式非常适合用于测试和快速验证想法。
"""

from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hello, Bob! It's nice to meet you. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 24, 'total_tokens': 43, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-ef14e0d8-d383-4bc9-b067-8a3d6612fb30-0', usage_metadata={'input_tokens': 24, 'output_tokens': 19, 'total_tokens': 43, 'input_token_details': {}, 'output_token_details': {}})

In [4]:
"""模型本身没有任何state的概念。例如，如果你问一个后续问题："""

model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our interaction or if you're one of the specific users who is associated with this platform and your name has been mentioned previously. Since AI interactions generally do not retain personal information for privacy reasons, I don't know your name unless you've introduced yourself recently during this conversation. You can tell me your name, though, which will help personalize our interaction!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 24, 'total_tokens': 120, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-e6e9959f-82fe-43f9-8f72-1b452d84963a-0', usage_metadata={'input_tokens': 24, 'output_tokens': 96, 'total_tokens': 120, 'input_token_details': {}, 'out

In [5]:
"""
我们可以看到，它并没有将之前的对话上下文考虑在内，因此无法回答问题。这样的聊天机器人体验非常糟糕！
为了解决这个问题，我们需要将整个对话历史传递给模型。让我们看看当我们这样做时会发生什么：
"""

from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 49, 'total_tokens': 55, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2', 'system_fingerprint': 'fp_ollama', 'finish_reason': 'stop', 'logprobs': None}, id='run-ed3803c6-a002-4fdb-986a-529f067e9610-0', usage_metadata={'input_tokens': 49, 'output_tokens': 6, 'total_tokens': 55, 'input_token_details': {}, 'output_token_details': {}})

现在我们可以看到得到了一个良好的响应！  
这是支撑聊天机器人进行对话互动的基本思路。那么，我们如何最好地实现这一点呢？  

消息持久化  
LangGraph实现了一个内置的持久化层，这使其成为支持多轮对话的聊天应用程序的理想选择。  
将我们的聊天模型包裹在一个最小的LangGraph应用程序中，可以让我们自动持久化消息历史记录，从而简化多轮应用程序的开发。  
LangGraph带有一个简单的内存检查点机制，我们在下面使用了它。详见其文档以获取更多细节，包括如何使用不同的持久化后端（例如SQLite或Postgres）。  

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [7]:
"""
我们还需要创建一个config，每次调用 runnable 时都要传递它。这个config包含了不是直接作为输入的一部分，但仍然有用的信息。
在这个例子中，我们想要包含一个 thread_id。
这使我们能够用单个应用程序支持多个对话线程，这是当您的应用程序有多个用户时的常见需求。每个对话线程都有一个唯一的标识符，这样就可以区分不同用户的对话历史，确保对话的准确性和隐私性。
"""

config = {"configurable": {"thread_id": "abc123"}}

In [8]:
"""现在我们可以用如下方式调用app"""

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hello, Bob! It's nice to meet you. How can I assist you today?


In [9]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob.


In [10]:
"""
我们的聊天机器人现在能记住关于我们的一些信息。如果我们更改配置以引用不同的 thread_id，我们可以看到它会重新开始对话。这样就能够支持不同用户之间的独立对话，每个用户都有自己独特的对话历史。
"""

config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

As you're addressing me directly, I can infer that you might be David. However, accurate names often require context or direct information about the individual in question. So, based on the basic information available here, my best guess would be "David". For more specific assistance, could you please provide additional details?


In [11]:
"""
然而，我们可以随时回到原始的对话（因为我们已经将其持久化到数据库中）。这意味着即使切换了 thread_id 并开始了新的对话，我们仍然可以通过引用原来的 thread_id 来访问和继续之前的对话。这种机制确保了用户对话历史的连续性和可访问性。
"""

config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob.


以上就是我们如何支持聊天机器人与多个用户进行对话的方式！通过为每个用户或对话线程分配唯一的标识符，并将对话历史持久化存储，聊天机器人能够维护和管理多个独立的对话流，从而有效地支持多用户环境。

In [12]:
"""对于异步支持，更新 call_model 节点使其成为一个异步函数（使用async关键字），并在调用应用程序时使用.ainvoke
这样可以确保在需要异步处理的场景中，比如与外部服务通信或执行长时间运行的任务时，能够更高效地进行操作。通过这种方式，您可以利用异步编程的优势来提高应用程序的响应速度和性能。
"""

# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As an AI, I don't have access to personalized information about individuals unless it has been shared with me in the course of our conversation. You haven't provided any personal details, so I don't know your name. What can I help you with today?


到目前为止，我们所做的只是在模型周围添加了一个简单的持久化层。我们可以通过引入提示模板使聊天机器人变得更加复杂和个性化。这样做可以让我们根据用户的具体需求定制对话体验，比如通过动态调整提示内容来引导对话方向或提供更相关的回复。

提示模板用于将原始用户输入转换成LLM可以处理的格式。在这个例子中，原始用户输入只是一个消息，我们将其传递给LLM。  
现在让我们使这个过程变得更复杂一些。首先，添加一个带有自定义指令的系统消息（但仍以消息作为输入）。接下来，除了消息之外，我们还会增加更多的输入。

In [13]:
"""
为了添加系统消息，我们将创建一个ChatPromptTemplate。我们将使用MessagesPlaceholder来传递所有的消息。具体来说：

我们会创建一个包含系统消息的ChatPromptTemplate，该系统消息含有定制的指令。
利用MessagesPlaceholder，我们可以将所有对话消息传递进去，以便于在提示中使用。

这样做的目的是为了让聊天机器人不仅能够处理简单的消息输入，还能够基于更复杂的上下文和指令进行回应，从而提供更加个性化和精确的回复。
"""

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [14]:
"""
我们现在可以更新我们的应用程序以包含这个模板：

通过将上述提示模板集成到应用程序中，我们可以使聊天机器人的对话能力更加复杂和个性化。这意味着，当用户与聊天机器人交互时，不仅可以处理简单的消息输入，还能基于预先设定的系统消息和动态输入内容生成更丰富、更相关的回复。这样做能够显著提升用户体验，使得对话更加自然流畅。具体实现时，需要确保将用户输入以及任何必要的额外信息正确地传递给提示模板，并通过该模板格式化后提供给语言模型。
"""

workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [15]:
"""我们以相同的方式调用应用程序："""

config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, ye be well met, Jack! What winds bring thee to these swashbuckling parts?


In [16]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name, matey, be Jim. A good sturdy name for a grand soul like thee!


In [17]:
"""太棒了！现在让我们使提示变得更加复杂一些。"""

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [18]:
"""注意，我们已经在提示中添加了一个新的language输入。我们的应用程序现在有两个参数——messages和language。我们应该更新我们应用程序的状态以反映这一点："""
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [19]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola! Soy tu asistente útil. ¿Cómo puedo ayudarte hoy, Bob?


In [20]:
"""注意，整个状态会被持久化，因此如果不需要更改，我们可以省略像language这样的参数："""

query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob.


管理对话历史  

在构建聊天机器人时，一个重要的概念是如何管理对话历史。如果不对对话历史进行管理，消息列表将无限制地增长，可能会超出LLM的上下文窗口。因此，添加一个步骤来限制传递的消息大小是很重要的。

In [21]:
"""
重要的是，你需要在应用提示模板之前，并且在从消息历史中加载先前消息之后进行这一步操作。

我们可以通过在提示前添加一个简单的步骤来适当地修改messages key，然后将这个new chain包裹在消息历史类中来实现这一点。

LangChain提供了一些内置的帮助函数来管理消息列表。在这个例子中，我们将使用trim_messages帮助函数来减少发送给模型的消息数量。这个trimmer允许我们指定想要保留多少tokens，以及其他参数，如是否总是保留系统消息以及是否允许部分消息：
"""

"""
token_counter设置为model，会出现get_num_tokens_from_messages() is not presently implemented for model cl100k_base错误。 
这是因为我们使用的模型qwen2不支持token个数计算。
一个解决方法是自定义token计数器（即下面的custom_token_counter函数），使用OpenAI的tiktoken库来计算token数量。
"""

from langchain_core.messages import SystemMessage, trim_messages

import tiktoken

def custom_token_counter(messages):
    encoding = tiktoken.get_encoding("cl100k_base")  # 使用 cl100k_base 编码
    total_tokens = 0
    for message in messages:
        total_tokens += len(encoding.encode(message.content))
    return total_tokens

trimmer = trim_messages(
    max_tokens=35,
    strategy="last",
    token_counter=custom_token_counter,  # 使用自定义的 token 计数器,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [22]:
"""为了在我们的chain中使用它，我们只需要在将message传递给prompt之前运行trimmer。"""

workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [23]:
"""现在，如果我们尝试问模型我们的名字，它将不知道，因为我们修剪了聊天历史的那部分"""

config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As an AI, I don't have information about individuals unless it was provided within the course of our conversation that you specifically mentioned your name. Without such specific input from you, I cannot determine or reveal your name. My responses are always respectful and adhere to privacy guidelines.


In [24]:
"""但如果我们就最近几条消息中的信息进行提问，它会记得"""

config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked, "What is the result of 2 + 2?" The answer to that question is 4.


流式传输  
现在我们已经拥有一个可以正常工作的聊天机器人。然而，对于聊天机器人应用程序来说，一个非常重要的用户体验考虑是流式传输。由于LLM有时需要一些时间来生成回复，大多数应用程序为了提升用户体验，会将每个生成的token即时流式传输回用户端。这使得用户能够看到进度。

In [25]:
"""
默认情况下，LangGraph应用程序中的.stream方法会进行流式传输，即模型响应这一步骤。设置stream_mode="messages"可以让我们改为流式传输输出的tokens：
"""

config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Sure|,| I|'d| be| happy| to| make| you| smile|,| Todd|!| Here|'s| a| classic| joke|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|Because| it| was| two|-t|ired|!

|I| hope| that| brought| a| smile| to| your| face|.| If| you| need| another| one| or| have| any| other| questions|,| feel| free| to| ask|!||